In [ ]:
# Install required libraries
!pip install --upgrade pip
!pip install opencv-python-headless numpy scikit-learn ultralytics matplotlib
!pip install ipywidgets
!pip install streamlit
!pip install tensorflow
!pip install tensorflow_hub
!pip install transformers
!pip install torch torchvision


In [ ]:
import glob

for file in glob.glob('videos/*.mp4'):
    print(file)

In [ ]:
from PIL import Image
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection 

In [ ]:
import torch

model_id = "IDEA-Research/grounding-dino-base"
device = "cuda" if torch.cuda.is_available() else "cpu"

processor = AutoProcessor.from_pretrained("models\Grounded-Dino-Model\processor")
model = AutoModelForZeroShotObjectDetection.from_pretrained("models\Grounded-Dino-Model\model").to(device)

In [ ]:
import requests
import torch
from PIL import Image

image_url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(image_url, stream=True).raw)
# Check for cats and remote controls
# VERY important: text queries need to be lowercased + end with a dot
text = "a cat. a remote control."

inputs = processor(images=image, text=text, return_tensors="pt").to(device)
with torch.no_grad():
    outputs = model(**inputs)

results = processor.post_process_grounded_object_detection(
    outputs,
    inputs.input_ids,
    box_threshold=0.4,
    text_threshold=0.3,
    target_sizes=[image.size[::-1]]
)

In [ ]:
print(results)

In [ ]:
from transformers import CLIPProcessor, CLIPModel

# Load the CLIP model for image embeddings
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")